### Import Libs

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import sqlite3
import ast

### Df Params

In [20]:
get_email = False
af_by_name = True

### Getting df from db

In [21]:
columns = [f'Mise {i}' for i in range(1,6)] + [i for i in range(1,58)]
query_columns = 'Mises, AF'
if get_email:
    columns = ['Email'] + columns
    query_columns = query_columns + ', email'

df = pd.DataFrame(columns = columns)
AF_dict = pd.read_pickle('Dicionario.pkl')

conn = sqlite3.connect("banco_de_dados.db")
cursor = conn.cursor()
cmd = f"SELECT {query_columns} from questionnaire_s8 ORDER BY ID ASC"
cursor.execute(cmd)
query_result = cursor.fetchall()

for i in range(len(query_result)):
    disciplinas = ast.literal_eval(query_result[i][1])
    if af_by_name: disciplinas = list(map(lambda x: AF_dict[x],disciplinas))
    
    if len(disciplinas) < 57:
        missing = 57 - len(disciplinas)
        disciplinas += ['-' for i in range(missing)]
    
    if get_email:
        row = pd.Series([query_result[i][2]] + ast.literal_eval(query_result[i][0]) +  disciplinas, index=df.columns)
    else:
        row = pd.Series(ast.literal_eval(query_result[i][0]) +  disciplinas, index=df.columns)

    df = df.append(row, ignore_index=True)

### Creating Visualization

In [56]:
dict_AF = {}

for index in df.index:
    subjects = df.loc[index, [i+1 for i in range(57)]]

    for subject_index in range(1, subjects.shape[0]+1):
        subject = subjects.loc[subject_index]

        if subject in dict_AF:
            dict_AF[subject].append(subject_index)
        else: 
            dict_AF[subject]=[subject_index]

In [62]:
dict_mises = {}

for index in df.index:
    mises = df.loc[index, [f'Mise {i+1}' for i in range(5)]]

    for position in range(1, mises.shape[0]+1):
        mise = mises.loc[f'Mise {position}']

        if f'Mise {position}' in dict_mises:
            dict_mises[f'Mise {position}'].append(mise)
        else: 
            dict_mises[f'Mise {position}']=[mise]


In [63]:
dict_mises

{'Mise 1': [30.0,
  34.0,
  24.0,
  30.2,
  25.0,
  30.0,
  41.0,
  37.5,
  41.0,
  30.5,
  25.0,
  30.0,
  50.0,
  27.0],
 'Mise 2': [27.0,
  31.0,
  24.0,
  26.0,
  23.0,
  26.0,
  35.0,
  33.5,
  31.0,
  25.5,
  20.0,
  30.0,
  15.0,
  23.0],
 'Mise 3': [20.0,
  25.0,
  24.0,
  22.0,
  20.0,
  26.0,
  11.0,
  22.5,
  21.0,
  20.5,
  20.0,
  15.0,
  15.0,
  21.0],
 'Mise 4': [13.0,
  7.0,
  14.0,
  21.7,
  17.0,
  10.0,
  10.0,
  5.5,
  4.0,
  12.5,
  20.0,
  15.0,
  15.0,
  20.0],
 'Mise 5': [10.0,
  3.0,
  14.0,
  0.1,
  15.0,
  8.0,
  3.0,
  1.0,
  3.0,
  11.0,
  15.0,
  10.0,
  5.0,
  9.0]}

In [52]:
tst.shape

(57,)